# Ovis VLM with RAG (Retrieval-Augmented Generation)

This notebook demonstrates how to add RAG capabilities to the Ovis Vision-Language Model. By integrating a retrieval system, the model can answer questions using information from an external knowledge base, leading to more accurate and detailed responses.

In [ ]:
# !pip install Pillow sentence-transformers pandas openpyxl chromadb langchain

In [ ]:
# Qwen/QwQ-32B 설치 (transformers, accelerate 등 필요)
!pip install --upgrade pip
!pip install transformers accelerate
# Qwen/QwQ-32B 모델은 transformers에서 자동 다운로드됨 (최초 실행 시 시간 소요)


## Step 1: Prepare Knowledge Base

First, we'll create a simple knowledge base. We'll use a few text documents, split them into manageable chunks, and then convert them into vector embeddings using a sentence transformer model. These embeddings will be stored in a FAISS index for fast retrieval.

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load the Excel file
try:
    print('Step 1: Reading Excel file...')
    df = pd.read_excel('/home/aisw/Project/UST-ETRI-2025/VLM_RAG/data/korean_train_20250101.xlsx')
    print('Step 2: Creating combined text columns...')
    # 모든 컬럼을 문자열로 합침
    all_columns = df.columns.tolist()
    df['임베딩텍스트'] = df[all_columns].astype(str).agg(' / '.join, axis=1)
    documents = df['임베딩텍스트'].tolist()
    print(f'Step 3: Total documents to embed: {len(documents)}')

    # (선택) 데이터 일부만 사용
    # documents = documents[:1000]

    print('Step 4: Loading embedding model...')
    embedding_model = SentenceTransformer('BAAI/bge-m3')
    import torch
    print('Step 5: Starting embedding (this may take a while)...')
    doc_embeddings = embedding_model.encode(documents, show_progress_bar=True, device='cuda' if torch.cuda.is_available() else 'cpu')
    print('Step 6: Embedding finished.')
    print(f"Successfully loaded and combined text from Excel file.")
    print("Document embeddings created successfully.")
    print("Shape of embeddings:", doc_embeddings.shape)

except FileNotFoundError:
    print("Error: korean_train_20250101.xlsx not found. Using sample data instead.")
    documents = [
        "광장시장은 대한민국 서울특별시 종로구에 위치한 전통 시장이다.",
        "1905년에 개설되었으며, 대한민국 최초의 상설 시장으로 알려져 있다.",
        "주요 판매 품목은 한복, 직물, 구제 의류, 그리고 다양한 먹거리이다.",
        "특히 빈대떡, 마약김밥, 육회 등이 유명하여 많은 관광객들이 찾는다."
    ]
    print('Step 4: Loading embedding model...')
    embedding_model = SentenceTransformer('BAAI/bge-m3')
    import torch
    print('Step 5: Starting embedding (sample data)...')
    doc_embeddings = embedding_model.encode(documents, show_progress_bar=True, device='cuda' if torch.cuda.is_available() else 'cpu')
    print('Step 6: Embedding finished.')
    print("Document embeddings created successfully.")
    print("Shape of embeddings:", doc_embeddings.shape)
except Exception as e:
    print(f"An error occurred: {e}. Using sample data.")
    documents = [
        "광장시장은 대한민국 서울특별시 종로구에 위치한 전통 시장이다.",
        "1905년에 개설되었으며, 대한민국 최초의 상설 시장으로 알려져 있다.",
        "주요 판매 품목은 한복, 직물, 구제 의류, 그리고 다양한 먹거리이다.",
        "특히 빈대떡, 마약김밥, 육회 등이 유명하여 많은 관광객들이 찾는다."
    ]
    print('Step 4: Loading embedding model...')
    embedding_model = SentenceTransformer('BAAI/bge-m3')
    import torch
    print('Step 5: Starting embedding (sample data)...')
    doc_embeddings = embedding_model.encode(documents, show_progress_bar=True, device='cuda' if torch.cuda.is_available() else 'cpu')
    print('Step 6: Embedding finished.')
    print("Document embeddings created successfully.")
    print("Shape of embeddings:", doc_embeddings.shape)

/home/aisw/Project/UST-ETRI-2025/VLM_RAG/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Step 1: Reading Excel file...


/home/aisw/Project/UST-ETRI-2025/VLM_RAG/venv/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Step 2: Creating combined text columns...
Step 3: Total documents to embed: 1070
Step 4: Loading embedding model...
Step 5: Starting embedding (this may take a while)...
Step 5: Starting embedding (this may take a while)...


Batches: 100%|██████████| 34/34 [00:02<00:00, 11.69it/s]

Step 6: Embedding finished.
Successfully loaded and combined text from Excel file.
Document embeddings created successfully.
Shape of embeddings: (1070, 1024)


In [2]:
import chromadb

# 프로젝트 내에 chroma_db 폴더에 영구 저장
persist_dir = './chroma_db'
client = chromadb.PersistentClient(path=persist_dir)

# Create a new collection or get an existing one
collection_name = "korean_knowledge_base"
collection = client.get_or_create_collection(name=collection_name)

# Generate IDs for each document
doc_ids = [str(i) for i in range(len(documents))]

# Add documents to the collection
collection.add(
    embeddings=doc_embeddings.tolist(),
    documents=documents,
    ids=doc_ids
)

print(f"ChromaDB collection '{collection_name}' created/updated with {collection.count()} documents. (persisted at {persist_dir})")

ChromaDB collection 'korean_knowledge_base' created/updated with 1070 documents. (persisted at ./chroma_db)


## Step 2: Implement Retriever

Now, we'll create a retriever function. This function will take a user's query, embed it using the same sentence transformer model, and then search the FAISS index to find the most relevant document chunks.

In [3]:
def retrieve_documents(query, k=2):
    # Embed the query
    query_embedding = embedding_model.encode([query]).tolist()
    
    # Query the collection
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=k
    )
    
    # Return the retrieved documents
    return results['documents'][0]

# Test the retriever with a Korean query
test_query = "평촌역의 위도와 경도를 알려주세요"
retrieved = retrieve_documents(test_query)
print(f"Query: {test_query}")
print("Retrieved documents:")
for doc in retrieved:
    print(f"- {doc}")

Query: 평촌역의 위도와 경도를 알려주세요
Retrieved documents:
- 코레일 / 4호선 / 도시/광역철도 / 441 / 평촌역 / Pyeongchon / Pyeongchon / ピョンチョン / 坪村 / 坪村 / nan / 불가능 / nan / 없음 / 없음 / 있음 / 있음 / 상대식 / 126.963881 / 37.394346 / nan / 경기도 안양시 동안구 부림로 지하 123 / 031-383-7788 / 19930115 / nan / 1.6 / 1.3 / 20231220 / nan / nan
- 코레일 / 경의중앙선 / 도시/광역철도 / P314 / 신촌 / Sinchon / Sinchon / シンチョン / 新村 / 新村 / nan / 불가능 / nan / 없음 / 없음 / 있음 / 있음 / 섬식 / 126.942308 / 37.559768 / 서울특별시 신촌동 74-12 / 서울특별시 서대문구 신촌역로 30 / 02-363-7788 / 2009-07-01 00:00:00 / nan / 3.1 / 2.7 / 20231220 / nan / nan


## Step 3: Load Ovis VLM Model

Next, we load the Ovis VLM model and its tokenizers. This code is adapted from your `main.py` script.

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image

model_path = "AIDC-AI/Ovis2-8B"

torch_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch_dtype,
    trust_remote_code=True,
    cache_dir="./hf_cache",
    device_map="auto",
    low_cpu_mem_usage=True,
)

tokenizer = model.get_text_tokenizer()
visual_tokenizer = model.get_visual_tokenizer()

print("Ovis VLM model and tokenizers loaded successfully.")

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.04it/s]



Ovis VLM model and tokenizers loaded successfully.


## Step 4: RAG-Enhanced Inference

Finally, we'll combine everything. We'll take an image and a question, use our retriever to find relevant information, construct a new prompt with this context, and then feed it to the Ovis model to get a RAG-enhanced answer.

In [ ]:
# Step 1: 이미지로부터 Ovis가 1차 설명 생성
import os
image_path = '/home/aisw/Project/UST-ETRI-2025/VLM_RAG/data/Pyeongchon_station.jpg'  # 실제 이미지 경로로 변경
image_path = '/home/aisw/Project/UST-ETRI-2025/VLM_RAG/data/seoul_station.jpg'  # 실제 이미지 경로로 변경


if not os.path.exists(image_path):
    print(f"Warning: Image not found at {image_path}. Please upload an image.")
    images = [Image.new('RGB', (512, 512), color='blue')]
else:
    images = [Image.open(image_path)]

# 1차 프롬프트: 이미지만 보고 설명 생성
image_only_prompt = "이 이미지를 보고 간단히 설명해 주세요. <image>"
max_partition = 9
prompt, input_ids, pixel_values = model.preprocess_inputs(image_only_prompt, images, max_partition=max_partition)
attention_mask = torch.ne(input_ids, tokenizer.pad_token_id)
input_ids = input_ids.unsqueeze(0).to(device=model.device)
attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
if pixel_values is not None:
    pixel_values = pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)
pixel_values = [pixel_values]

with torch.inference_mode():
    gen_kwargs = dict(max_new_tokens=256, do_sample=False)
    output_ids = model.generate(input_ids, pixel_values=pixel_values, attention_mask=attention_mask, **gen_kwargs)[0]
    image_description = tokenizer.decode(output_ids, skip_special_tokens=True)

print("--- 이미지 설명 ---")
print(image_description)

# Step 2: 이미지 설명을 기반으로 벡터DB에서 관련 정보 검색
k = 3
query_embedding = embedding_model.encode([image_description], show_progress_bar=False, device='cuda' if torch.cuda.is_available() else 'cpu')
results = collection.query(query_embeddings=query_embedding.tolist(), n_results=k)
retrieved_context = results['documents'][0]
context_str = "\n".join(retrieved_context)

# Step 3: 최종 RAG 프롬프트 생성 및 응답
user_question = "이 역에서 어떤 노선으로 환승할 수 있나요?"   # 실제 질문으로 교체 가능

# --- 프롬프트 더욱 엄격하게 보완 ---
rag_prompt = f"""
아래 [문맥]에 주어진 정보만을 근거로 [질문]에 답변하세요.
- 반드시 [문맥]의 내용을 직접 인용하거나 요약하여 답변하세요.
- [문맥]에 답이 없거나 불충분하면, 반드시 아래 예시처럼만 답변하세요:
  'VectorDB(지식베이스)에서 답변을 찾을 수 없습니다.'
- [문맥]에 없는 내용을 상상하거나 지어내지 마세요. 추가 설명도 하지 마세요.
- 예시)
  [문맥]이 비어있거나 관련 정보가 없을 때: 'VectorDB(지식베이스)에서 답변을 찾을 수 없습니다.'

[문맥]
{context_str if context_str.strip() else '없음'}

[질문]
{user_question}
<image>
"""

prompt, input_ids, pixel_values = model.preprocess_inputs(rag_prompt, images, max_partition=max_partition)
attention_mask = torch.ne(input_ids, tokenizer.pad_token_id)
input_ids = input_ids.unsqueeze(0).to(device=model.device)
attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
if pixel_values is not None:
    pixel_values = pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)
pixel_values = [pixel_values]

with torch.inference_mode():
    gen_kwargs = dict(max_new_tokens=1024, do_sample=False)
    output_ids = model.generate(input_ids, pixel_values=pixel_values, attention_mask=attention_mask, **gen_kwargs)[0]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)

print("--- RAG 기반 답변 ---")
print(f'결과:\n{output}')

In [ ]:
# Step 1: 이미지로부터 Ovis가 1차 설명 생성
import os
image_path = '/home/aisw/Project/UST-ETRI-2025/VLM_RAG/data/Daejeon_station.jpeg'  # 실제 이미지 경로로 변경


if not os.path.exists(image_path):
    print(f"Warning: Image not found at {image_path}. Please upload an image.")
    images = [Image.new('RGB', (512, 512), color='blue')]
else:
    images = [Image.open(image_path)]

# 1차 프롬프트: 이미지만 보고 설명 생성
image_only_prompt = "이 이미지를 보고 간단히 설명해 주세요. <image>"
max_partition = 9
prompt, input_ids, pixel_values = model.preprocess_inputs(image_only_prompt, images, max_partition=max_partition)
attention_mask = torch.ne(input_ids, tokenizer.pad_token_id)
input_ids = input_ids.unsqueeze(0).to(device=model.device)
attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
if pixel_values is not None:
    pixel_values = pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)
pixel_values = [pixel_values]

with torch.inference_mode():
    gen_kwargs = dict(max_new_tokens=256, do_sample=False)
    output_ids = model.generate(input_ids, pixel_values=pixel_values, attention_mask=attention_mask, **gen_kwargs)[0]
    image_description = tokenizer.decode(output_ids, skip_special_tokens=True)

print("--- 이미지 설명 ---")
print(image_description)

# Step 2: 이미지 설명을 기반으로 벡터DB에서 관련 정보 검색
k = 3
query_embedding = embedding_model.encode([image_description], show_progress_bar=False, device='cuda' if torch.cuda.is_available() else 'cpu')
results = collection.query(query_embeddings=query_embedding.tolist(), n_results=k)
retrieved_context = results['documents'][0]
context_str = "\n".join(retrieved_context)

# Step 3: 최종 RAG 프롬프트 생성 및 응답
user_question = "대전역의 역사 전화번호는 어떻게 되나요?"   # 실제 질문으로 교체 가능

# --- 프롬프트 더욱 엄격하게 보완 ---
rag_prompt = f"""
아래 [문맥]에 주어진 정보만을 근거로 [질문]에 답변하세요.
- 반드시 [문맥]의 내용을 직접 인용하거나 요약하여 답변하세요.
- [문맥]에 답이 없거나 불충분하면, 반드시 아래 예시처럼만 답변하세요:
  'VectorDB(지식베이스)에서 답변을 찾을 수 없습니다.'
- [문맥]에 없는 내용을 상상하거나 지어내지 마세요. 추가 설명도 하지 마세요.
- 예시)
  [문맥]이 비어있거나 관련 정보가 없을 때: 'VectorDB(지식베이스)에서 답변을 찾을 수 없습니다.'

[문맥]
{context_str if context_str.strip() else '없음'}

[질문]
{user_question}
<image>
"""

prompt, input_ids, pixel_values = model.preprocess_inputs(rag_prompt, images, max_partition=max_partition)
attention_mask = torch.ne(input_ids, tokenizer.pad_token_id)
input_ids = input_ids.unsqueeze(0).to(device=model.device)
attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
if pixel_values is not None:
    pixel_values = pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)
pixel_values = [pixel_values]

with torch.inference_mode():
    gen_kwargs = dict(max_new_tokens=1024, do_sample=False)
    output_ids = model.generate(input_ids, pixel_values=pixel_values, attention_mask=attention_mask, **gen_kwargs)[0]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)

print("--- RAG 기반 답변 ---")
print(f'결과:\n{output}')

In [10]:
# Step 1: 이미지로부터 Ovis가 1차 설명 생성
import os
image_path = '/home/aisw/Project/UST-ETRI-2025/VLM_RAG/data/test1.jpg'  # 실제 이미지 경로로 변경


if not os.path.exists(image_path):
    print(f"Warning: Image not found at {image_path}. Please upload an image.")
    images = [Image.new('RGB', (512, 512), color='blue')]
else:
    images = [Image.open(image_path)]

# 1차 프롬프트: 이미지만 보고 설명 생성
image_only_prompt = "이 이미지를 보고 간단히 설명해 주세요. <image>"
max_partition = 9
prompt, input_ids, pixel_values = model.preprocess_inputs(image_only_prompt, images, max_partition=max_partition)
attention_mask = torch.ne(input_ids, tokenizer.pad_token_id)
input_ids = input_ids.unsqueeze(0).to(device=model.device)
attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
if pixel_values is not None:
    pixel_values = pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)
pixel_values = [pixel_values]

with torch.inference_mode():
    gen_kwargs = dict(max_new_tokens=256, do_sample=False)
    output_ids = model.generate(input_ids, pixel_values=pixel_values, attention_mask=attention_mask, **gen_kwargs)[0]
    image_description = tokenizer.decode(output_ids, skip_special_tokens=True)

print("--- 이미지 설명 ---")
print(image_description)

# Step 2: 이미지 설명을 기반으로 벡터DB에서 관련 정보 검색
k = 3
query_embedding = embedding_model.encode([image_description], show_progress_bar=False, device='cuda' if torch.cuda.is_available() else 'cpu')
results = collection.query(query_embeddings=query_embedding.tolist(), n_results=k)
retrieved_context = results['documents'][0]
context_str = "\n".join(retrieved_context)

# Step 3: 최종 RAG 프롬프트 생성 및 응답
user_question = "현재 몇호선 라인을 타고있다고 추측할 수 있나요? 그 이유가 뭔가요?"   # 실제 질문으로 교체 가능

# --- 프롬프트 더욱 엄격하게 보완 ---
rag_prompt = f"""
당신은 지하철 정보에 특화된 질의응답 전문가입니다.
당신은 이미지 분석 후 생성된 문서를 기반으로 정확하고 검증된 정보를 제공하는 지식 기반 응답 전문가입니다.
아래 [문맥]에 주어진 정보만을 근거로 [질문]에 답변하세요.
- 반드시 [문맥]의 내용을 직접 인용하거나 요약하여 답변하세요.
- [문맥]에 답이 없거나 불충분하면, 반드시 아래 예시처럼만 답변하세요:
  'VectorDB(지식베이스)에서 답변을 찾을 수 없습니다.'
- [문맥]에 없는 내용을 상상하거나 지어내지 마세요. 추가 설명도 하지 마세요.
- 예시)
  [문맥]이 비어있거나 관련 정보가 없을 때: 'VectorDB(지식베이스)에서 답변을 찾을 수 없습니다.'

[문맥]
{context_str if context_str.strip() else '없음'}

[질문]
{user_question}
<image>
"""

prompt, input_ids, pixel_values = model.preprocess_inputs(rag_prompt, images, max_partition=max_partition)
attention_mask = torch.ne(input_ids, tokenizer.pad_token_id)
input_ids = input_ids.unsqueeze(0).to(device=model.device)
attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
if pixel_values is not None:
    pixel_values = pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)
pixel_values = [pixel_values]

with torch.inference_mode():
    gen_kwargs = dict(max_new_tokens=1024, do_sample=False)
    output_ids = model.generate(input_ids, pixel_values=pixel_values, attention_mask=attention_mask, **gen_kwargs)[0]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)

print("--- RAG 기반 답변 ---")
print(f'결과:\n{output}')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


--- 이미지 설명 ---
이 이미지는 한국의 지하철 내부를 보여줍니다. 상단에는 시간이 11:52로 표시되어 있으며, "신내행"이라는 목적지와 "U+5G 갤러리 공덕역 오픈"이라는 문구가 눈에 띕니다. 또한, "6호선 공덕역 플랫폼 및 환승공간"이라는 안내문이 있습니다. 오른쪽에는 지하철 내부의 모습이 담긴 홍보 영상이 재생되고 있습니다. 하단에는 "개인회생·파산면책"이라는 문구와 함께 "1811-1890"이라는 전화번호가 표시되어 있습니다. 이는 개인회생 및 파산면책 상담을 위한 전화번호입니다. 또한, 지하철 노선도와 함께 "무료 상담"이라는 문구가 보입니다.
--- RAG 기반 답변 ---
결과:
현재 타고 있는 라인은 5호선입니다. 이는 디지털 표시판에 '공덕' 역이 표시되어 있고, 'U+5G 갤러리 공덕역 오픈'이라는 문구가 보이기 때문입니다. 또한, 지하철 내부의 광고판에 '5%'라는 숫자가 표시되어 있어 5호선을 타고 있다는 것을 추측할 수 있습니다.
--- RAG 기반 답변 ---
결과:
현재 타고 있는 라인은 5호선입니다. 이는 디지털 표시판에 '공덕' 역이 표시되어 있고, 'U+5G 갤러리 공덕역 오픈'이라는 문구가 보이기 때문입니다. 또한, 지하철 내부의 광고판에 '5%'라는 숫자가 표시되어 있어 5호선을 타고 있다는 것을 추측할 수 있습니다.


In [19]:
# Step 1: 이미지로부터 Ovis가 1차 설명 생성
import os
image_path = '/home/aisw/Project/UST-ETRI-2025/VLM_RAG/data/where.png'  # 실제 이미지 경로로 변경


if not os.path.exists(image_path):
    print(f"Warning: Image not found at {image_path}. Please upload an image.")
    images = [Image.new('RGB', (512, 512), color='blue')]
else:
    images = [Image.open(image_path)]

# 1차 프롬프트: 이미지만 보고 설명 생성
image_only_prompt = "이 이미지를 보고 간단히 설명해 주세요. <image>"
max_partition = 9
prompt, input_ids, pixel_values = model.preprocess_inputs(image_only_prompt, images, max_partition=max_partition)
attention_mask = torch.ne(input_ids, tokenizer.pad_token_id)
input_ids = input_ids.unsqueeze(0).to(device=model.device)
attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
if pixel_values is not None:
    pixel_values = pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)
pixel_values = [pixel_values]

with torch.inference_mode():
    gen_kwargs = dict(max_new_tokens=1024, do_sample=False)
    output_ids = model.generate(input_ids, pixel_values=pixel_values, attention_mask=attention_mask, **gen_kwargs)[0]
    image_description = tokenizer.decode(output_ids, skip_special_tokens=True)

print("--- 이미지 설명 ---")
print(image_description)

# Step 2: 이미지 설명을 기반으로 벡터DB에서 관련 정보 검색
k = 3
query_embedding = embedding_model.encode([image_description], show_progress_bar=False, device='cuda' if torch.cuda.is_available() else 'cpu')
results = collection.query(query_embeddings=query_embedding.tolist(), n_results=k)
retrieved_context = results['documents'][0]
context_str = "\n".join(retrieved_context)
print(retrieved_context)

# Step 3: 최종 RAG 프롬프트 생성 및 응답
user_question = "지금 보고 있는 전광판은 몇호선에 대한 전광판인가요?"   # 실제 질문으로 교체 가능

# --- 프롬프트 더욱 엄격하게 보완 ---
rag_prompt = f"""
당신은 지하철 정보에 특화된 질의응답 전문가입니다.
당신은 이미지 분석 후 생성된 문서를 기반으로 정확하고 검증된 정보를 제공하는 지식 기반 응답 전문가입니다.
아래 [문맥]에 주어진 정보만을 근거로 [질문]에 답변하세요.
이유도 필히 포함하여 답변하세요.
- 반드시 [문맥]의 내용을 직접 인용하거나 요약하여 답변하세요.
- [문맥]에 답이 없거나 불충분하면, 반드시 아래 예시처럼만 답변하세요:
  'VectorDB(지식베이스)에서 답변을 찾을 수 없습니다.'
- [문맥]에 없는 내용을 상상하거나 지어내지 마세요. 추가 설명도 하지 마세요.
- 예시)
  [문맥]이 비어있거나 관련 정보가 없을 때: 'VectorDB(지식베이스)에서 답변을 찾을 수 없습니다.'

[문맥]
{context_str if context_str.strip() else '없음'}

[질문]
{user_question}
<image>
"""

prompt, input_ids, pixel_values = model.preprocess_inputs(rag_prompt, images, max_partition=max_partition)
attention_mask = torch.ne(input_ids, tokenizer.pad_token_id)
input_ids = input_ids.unsqueeze(0).to(device=model.device)
attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
if pixel_values is not None:
    pixel_values = pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)
pixel_values = [pixel_values]

with torch.inference_mode():
    gen_kwargs = dict(max_new_tokens=1024, do_sample=False)
    output_ids = model.generate(input_ids, pixel_values=pixel_values, attention_mask=attention_mask, **gen_kwargs)[0]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)

print("--- RAG 기반 답변 ---")
print(f'결과:\n{output}')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


--- 이미지 설명 ---
이 이미지는 한국의 지하철역 내부에서 촬영된 것으로 보입니다. 상단에는 "영구·지하철 연락전용 화면"이라는 제목이 적힌 디지털 표지판이 보입니다. 표지판에는 다음 열차의 출발 정보가 표시되어 있습니다.

1. **표지판 내용**:
   - **출발 순서**: 1번, 2번, 3번
   - **열차 번호**: 1호열차
   - **행선지**: 덕소, 용문, 덕소
   - **출발 시간**: 12:26, 12:41, 12:53
   - **상태**: 출발 예정 (출발 시간이 빨간색으로 표시되어 있음)

2. **배경**:
   - **지하철역 내부**: 천장에 설치된 디지털 표지판과 지하철역 내부의 일반적인 구조가 보입니다.
   - **안내 표지판**: 표지판 아래에는 "행신·일산·문산"과 "서울역·신촌·화전"으로 이동하는 방향을 가리키는 파란색 안내 표지판이 있습니다.

이 이미지는 지하철역 내부에서 열차의 출발 정보를 확인할 수 있는 디지털 표지판을 보여주고 있습니다.
['서울교통공사 / 4호선 / 도시/광역철도 / 426 / 서울역 / Seoul Station / nan / ソウルヨク / 首尔站 / nan / nan / O / 1호선, 공항철도, 경의중앙선 / 없음 / 있음 / 있음 / nan / 섬식 / 126.972836 / 37.553172 / 서울특별시 용산구 동자동 14-151 서울역 (4호선) / 서울특별시 용산구 한강대로 지하392(동자동) / 02-6110-4261 / 19851018 / nan / 0.8999999999999986 / 0.8999999999999986 / 20241231 / nan / nan', '대구교통공사 / 2호선 / 도시/광역철도 / 0220 / 계명대 / Keimyung Univ. / 없음 / 없음 / 없음 / 啓明大 / 없음 / 불가능 / 없음 / 없음 / 있음 / 있음 / 없음 / 상대식 / 128.49196903 / 35.85146491 / - / 대구광역시 달서구 달구벌대로 지하114

In [2]:
# LangChain 기반 이미지→설명→문맥검색→답변 체인 (Qwen/QwQ-32B 활용)
import os
from PIL import Image
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM
import torch

# 1. 임베딩 모델 준비 (sentence-transformers 호환)
embedding_model_name = 'BAAI/bge-m3'
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 2. ChromaDB 벡터스토어 연결 (이미 생성된 chroma_db 사용)
persist_dir = './chroma_db'
vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 3. LLM 준비 (Qwen/QwQ-32B로 변경)
llm_pipe = pipeline(
    "text-generation",
    model="Qwen/QwQ-32B",
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto",
    max_new_tokens=1024,
)
llm = HuggingFacePipeline(pipeline=llm_pipe)

# 4. Ovis VLM 모델 및 토크나이저 로드 (이미지 설명 생성용)
model_path = "AIDC-AI/Ovis2-8B"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    trust_remote_code=True,
    cache_dir="./hf_cache",
    device_map="auto",
    low_cpu_mem_usage=True,
)
tokenizer = model.get_text_tokenizer()
visual_tokenizer = model.get_visual_tokenizer()

# 5. 이미지→설명→문맥검색→답변 함수
def langchain_image_rag(image_path, user_question):
    # (1) 이미지 로드 및 Ovis VLM 설명 생성
    if not os.path.exists(image_path):
        print(f"Warning: Image not found at {image_path}. Using blank image.")
        images = [Image.new('RGB', (512, 512), color='blue')]
    else:
        images = [Image.open(image_path)]
    image_only_prompt = "이 이미지를 보고 간단히 설명해 주세요. <image>"
    max_partition = 9
    prompt, input_ids, pixel_values = model.preprocess_inputs(image_only_prompt, images, max_partition=max_partition)
    attention_mask = torch.ne(input_ids, tokenizer.pad_token_id)
    input_ids = input_ids.unsqueeze(0).to(device=model.device)
    attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
    if pixel_values is not None:
        pixel_values = pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)
    pixel_values = [pixel_values]
    with torch.inference_mode():
        gen_kwargs = dict(max_new_tokens=256, do_sample=False)
        output_ids = model.generate(input_ids, pixel_values=pixel_values, attention_mask=attention_mask, **gen_kwargs)[0]
        image_description = tokenizer.decode(output_ids, skip_special_tokens=True)
    print("--- 이미지 설명 ---")
    print(image_description)

    # (2) 이미지 설명으로 벡터DB 검색
    retrieved_docs = retriever.get_relevant_documents(image_description)
    context_str = "\n".join([doc.page_content for doc in retrieved_docs])

    # (3) LangChain RAG QA 체인 실행 (프롬프트 엄격화)
    prompt_template = (
        "아래 [문맥]에 주어진 정보만을 근거로 [질문]에 답변하세요.\n"
        "- 반드시 [문맥]의 내용을 직접 인용하거나 요약하여 답변하세요.\n"
        "- [문맥]에 답이 없거나 불충분하면, 반드시 아래 예시처럼만 답변하세요:\n"
        "  'VectorDB(지식베이스)에서 답변을 찾을 수 없습니다.'\n"
        "- [문맥]에 없는 내용을 상상하거나 지어내지 마세요. 추가 설명도 하지 마세요.\n"
        "- 예시) [문맥]이 비어있거나 관련 정보가 없을 때: 'VectorDB(지식베이스)에서 답변을 찾을 수 없습니다.'\n\n"
        "[문맥]\n{context}\n\n[질문]\n{question}\n"
    )
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt_template}
    )
    # (4) 답변 생성
    result = qa_chain({"query": user_question, "context": context_str})
    print("--- LangChain 기반 RAG 답변 (Qwen/QwQ-32B) ---")
    print(result["result"])
    return result["result"]

# 사용 예시
image_path = '/home/aisw/Project/UST-ETRI-2025/VLM_RAG/data/where.png'  # 실제 이미지 경로로 변경
user_question = "지금 보고 있는 전광판은 몇호선에 대한 전광판인가요?"   # 실제 질문으로 교체 가능
langchain_image_rag(image_path, user_question)

/home/aisw/Project/UST-ETRI-2025/VLM_RAG/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_446985/2983381715.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
/tmp/ipykernel_446985/2983381715.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langcha

KeyboardInterrupt: 

In [1]:
# 4. Qwen/QwQ-32B LLM 준비 (transformers pipeline)
qwen_pipe = pipeline(
    "text-generation",
    model="Qwen/QwQ-32B",
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto",
    max_new_tokens=1024,
)

NameError: name 'pipeline' is not defined